# SN2009ip

In [308]:
import matplotlib.pyplot as plt
import os
import numpy as np
from astropy.table import Table, Column
import astropy.units as u
import pycoco as pcc
import json

Change SN_name to retrieve data from that super nova provided its stored in the data file

In [346]:
SN_folder = "/Users/bertiepwhite/code/Data/"    #The location where the .jsonn files are stored before handling
SN_name = "SN2009ip"                             #Current Supernova being worked on 
SN_file_name = SN_name + ".json"                #Produces the file name
path = os.path.join(SN_folder,SN_file_name)     #Produces the file location

Creating the .dat file if its not already created

In [ ]:
dat_file_name = SN_name +'.dat'
file_location = SN_folder + dat_file_name
print(dat_file_name,file_location)
f = open(file_location,'w+')
f.close()

Retrieving the data and splitting into 2 ditionaries one for photometry (ph) one for spectra (sp)

In [347]:
data = json.load(open(path,'r'))                #Grabs the file

ph_dict = data[SN_name]["photometry"]           #Spits the file into the 2 components
sp_dict = data[SN_name]["spectra"]              

ph_dict[3] #This is included to check the general form is as expected

{'band': 'V',
 'e_magnitude': '0.37',
 'magnitude': '20.99',
 'source': '10,19',
 'time': '55080.42',
 'u_time': 'MJD'}

Generating a list of the filters used

In [348]:
j = 0
filter_list = []

for i in ph_dict:
    if ph_dict[j]['magnitude'] != 0:
        band = ph_dict[j]['band']
        if band not in filter_list:
            filter_list.append(band)
        j += 1
filter_list

['R', 'I', 'V', 'W1', 'U', 'B', 'W2', 'M2', 'H', 'J', 'KS']

Producing a list of the full names corrosponding to the list above

In [366]:
filter_names=['Blank','BessellB','BessellV','BessellR','BessellI', 'Blank', 'Blank', 'Blank', 'Blank', 'Blank', 'Blank', 'Blank']
filter_short=['C', 'B', 'V', 'R', 'I', 'W1', 'U', 'W2', 'M2', 'H', 'J', 'KS']

In [374]:
filt_dict = {"B" : "BessellB",
            }

In [377]:
# key = "B"
key = "banana"
if key in filt_dict.keys():
    new_filter = filt_dict[key]
else:
    print("nope")
    new_filter = "not found"
print(new_filter)

nope
not found


Generating the data columns, We see they take the form Band, magnitude, errror, time

In [350]:
data_rows = []
j = 0
data_rows.append([' ',' ',' ','Days']) #Setting the units (FIND A BETTER WAY)

for a in ph_dict:
    data_entry = []
    
    #Using the band shorthand given and converting that into the full name
    band = (ph_dict[j]['band'])
    index = filter_list.index(band)
    data_entry.append(filter_names[index])
    
    data_entry.append(ph_dict[j]['magnitude']) #Simply apppending the magnitude
    
    try:
        data_entry.append(ph_dict[j]['e_magnitude']) #Appending the error on the magnitude and putting n/a 
    except KeyError:                                 #if that information is unavailable
        data_entry.append("n/a")
        
        
    data_entry.append(ph_dict[j]['time']) #Appending the time into the row
    data_rows.append(data_entry) #Appending the data entry into the data
    
    j += 1
    
    

In [362]:
data_entry

['BessellV', '21.73', '0.07', '57014.06']

In [385]:
t = Table(rows = data_rows, names = ('Band','Magnitude','E_Magnitude', 'Time'))
t['Time'].units='days' 

t.show_in_notebook()

idx,Band,Magnitude,E_Magnitude,Time
0,,,,Days
1,Blank,18.13,0.09,55073.6
2,BessellB,18.11,0.13,55073.64
3,BessellV,18.51,0.18,55073.74
4,BessellV,20.99,0.37,55080.42
5,Blank,20.43,0.03,55081.06
6,BessellV,21.06,0.05,55081.06
7,BessellV,20.7,0.17,55081.5
8,BessellV,20.73,0.2,55081.52
9,Blank,19.78,0.24,55081.53


In [353]:
ph_dict[0]

{'band': 'R',
 'e_magnitude': '0.09',
 'magnitude': '18.13',
 'source': '10,19',
 'time': '55073.6',
 'u_time': 'MJD'}

In [361]:
filter_file_location = "/Users/bertiepwhite/code/coco/data/filters/"
j = 0
filter_obj_list = []
AB = pcc.kcorr.load_AB()
vega = pcc.kcorr.load_vega()

for i in filter_names:
    filter_name = filter_names[j]
    if filter_name != "Blank":
        filter_obj_location = filter_file_location + filter_name +".dat"
        filter_obj = pcc.kcorr.load_filter(filter_obj_location)
        filter_obj_list .append(filter_obj)
        print(j)
    else:
        filter_obj_list .append("Filter N/A")
    print(filter_obj_list[-1])
    j += 1


j = 1
for i in range(len(data_rows)-1):
    filter_name = data_rows[j][0] 
    if filter_name != 'Blank':
        filter_index = filter_names.index(filter_name)
        filter_obj = filter_obj_list[index]
        
        if filter_obj != "Filter N/A":
            print("Working")
            filter_obj.resample_response(new_wavelength = AB.wavelegnth)

            transmitted_spec = filter_obj.throughput * AB.flux
            integrated_flux = simps(transmitted_spec,AB.wavelgnth)

            print(integrated_flux)
        j += 1

Filter N/A
1
2
3
4
Filter N/A
Filter N/A
Filter N/A
Filter N/A
Filter N/A
Filter N/A
Filter N/A
